Побудувати інформаційну панель (dashboard) використовуючи Tableau/Python та LTV отриманий у завданні 1.1:

- ROMI поденно для кожного mediasource з можливістю фільтрації по країнах
- ROMI поденно у розрізі топ-10 країн (топ вибираємо по витратах (cost) на маркетинг за весь час)
- Топ-10 комбінацій mediasource та country з найвищим ROMI для кожного календарного місяця

In [1]:
import pandas as pd

In [2]:
transactions = pd.read_csv('input_files/file1_transactions.csv')
expenses = pd.read_csv('input_files/file2_expenses.csv')

transactions['purchase_date'] = pd.to_datetime(transactions['purchase_date'])
expenses['date'] = pd.to_datetime(expenses['date'])

Для зручності розрахунків додамо 2 стовпця до датафрейму з транзакціями:

* Стовпець 'transaction_number' - нумерація оплат за часом для одного користувача
* Стовпець 'earnings' - прибуток для кожної операції

Так як інвестиції в маркетинг для всього бізнесу, а не для конкретного типу підписки, аналізуємо ROMI для всіх типів підписок. Показник ROMI будемо розраховувати за кумулятивними витратами та прибутком, адже беручи дані просто за окремий день ми не отримаємо релевантних результатів для оцінки вкладень в маркетинг.
 
Для підрахунку прибутку враховуємо, що для щотижневих підписок ціна пробного періоду відрізняється від подальшої.

In [3]:
transactions = transactions.sort_values(by=['user_id', 'purchase_date'])
transactions['transaction_number'] = transactions.groupby('user_id').cumcount() + 1

def calculate_earnings(row):
    if row['refunded']:
        return 0
    if row['product_id'] == 'tenwords_1w_7.99_7free':
        return 0 if row['transaction_number'] == 1 else 7.99
    elif row['product_id'] == 'tenwords_1w_9.99_offer':
        return 0.5 if row['transaction_number'] == 1 else 9.99
    elif row['product_id'] == 'tenwords_lifetime_limited_49.99':
        return 49.99
    else:
        return 0

transactions['earnings'] = transactions.apply(calculate_earnings, axis=1)
transactions.head(10)

,user_id,product_id,refunded,purchase_date,country_code,media_source,transaction_number,earnings
57919,3935,tenwords_lifetime_limited_49.99,False,2020-09-22,Organic/Unknown,Organic/Unknown,1,49.99
48466,12070,tenwords_1w_7.99_7free,False,2020-07-11,HR,Facebook Ads,1,0.00
44420,12254,tenwords_1w_7.99_7free,False,2020-09-06,US,Facebook Ads,1,0.00
6970,27637,tenwords_1w_9.99_offer,False,2020-07-05,PL,Facebook Ads,1,0.50
38650,31549,tenwords_1w_9.99_offer,False,2020-08-25,CO,Facebook Ads,1,0.50
61235,31549,tenwords_1w_9.99_offer,False,2020-08-25,CO,Facebook Ads,2,9.99
47316,31921,tenwords_1w_9.99_offer,False,2020-08-25,FR,Facebook Ads,1,0.50
31438,34244,tenwords_1w_9.99_offer,False,2020-09-07,NL,Facebook Ads,1,0.50
47105,34372,tenwords_1w_7.99_7free,False,2020-07-02,AE,Facebook Ads,1,0.00
45838,34491,tenwords_1w_9.99_offer,False,2020-07-15,Organic/Unknown,Organic/Unknown,1,0.50


In [4]:
expenses.head(10)

,date,country_code,cost,media_source
0,2020-07-01,AD,0.17,Facebook Ads
1,2020-07-01,AE,59.30,Facebook Ads
2,2020-07-01,AF,1.23,Facebook Ads
3,2020-07-01,AG,0.23,Facebook Ads
4,2020-07-01,AI,0.18,Facebook Ads
5,2020-07-01,AL,1.60,Facebook Ads
6,2020-07-01,AM,2.07,Facebook Ads
7,2020-07-01,AO,2.63,Facebook Ads
8,2020-07-01,AQ,0.01,Facebook Ads
9,2020-07-01,AR,6.73,Facebook Ads


Так як може бути багато записів з одними і тими ж значеннями 'date' ('purchase_date'), 'country_code', 'media_source', згрупуємо дані за цими трьома стовпцями і обрахуємо для них значення прибутку та витрат на маркетинг.

In [5]:
aggregated_transactions = transactions.groupby(
    ['purchase_date', 'country_code', 'media_source']
).agg({'earnings': 'sum', 'user_id': 'count'}).reset_index()

aggregated_transactions = aggregated_transactions.rename(columns={'user_id': 'user_count'})
aggregated_transactions.head()

,purchase_date,country_code,media_source,earnings,user_count
0,2020-07-01,AE,Facebook Ads,1.00,3
1,2020-07-01,AT,Facebook Ads,0.50,1
2,2020-07-01,AU,Facebook Ads,111.97,9
3,2020-07-01,BE,Facebook Ads,0.50,1
4,2020-07-01,BM,Facebook Ads,49.99,1


In [6]:
aggregated_expenses = expenses.groupby(
    ['date', 'country_code', 'media_source']
).agg({'cost': 'sum'}).reset_index()

aggregated_expenses.head()

,date,country_code,media_source,cost
0,2020-07-01,AD,Facebook Ads,0.17
1,2020-07-01,AE,Facebook Ads,59.30
2,2020-07-01,AF,Facebook Ads,1.23
3,2020-07-01,AG,Facebook Ads,0.23
4,2020-07-01,AI,Facebook Ads,0.18


Для аналізу ROMI об'єднаємо два датафрейми за трьома стовпцями. Якщо за певну дату та mediasource немає даних про витрати на маркетинг та прибуток, вважатимемо, що вони були рівні 0.

In [7]:
merged_data = aggregated_transactions.merge(
    aggregated_expenses,
    left_on=['purchase_date', 'country_code', 'media_source'],
    right_on=['date', 'country_code', 'media_source'],
    how='outer'
)

merged_data['earnings'] = merged_data['earnings'].fillna(0)
merged_data['user_count'] = merged_data['user_count'].fillna(0)

merged_data['date'] = merged_data['date'].fillna(merged_data['purchase_date'])
merged_data['purchase_date'] = merged_data['purchase_date'].fillna(merged_data['date'])
merged_data = merged_data.drop('purchase_date', axis=1)

merged_data.head()

,country_code,media_source,earnings,user_count,date,cost
0,AE,Facebook Ads,1.00,3.0,2020-07-01,59.30
1,AT,Facebook Ads,0.50,1.0,2020-07-01,7.53
2,AU,Facebook Ads,111.97,9.0,2020-07-01,73.72
3,BE,Facebook Ads,0.50,1.0,2020-07-01,13.26
4,BM,Facebook Ads,49.99,1.0,2020-07-01,0.92


In [8]:
merged_data.to_csv('output_files/merged_data.csv', index=False)

#### ROMI поденно для кожного mediasource

Для обрахунку ROMI поденно для кожного mediasource, згрупуємо дані за 'media_source' та 'date'. Так як ми не маємо інвестицій для mediasource зі значенням Organic/Unknown, то для аналізу повернення маркетингових інвестицій з окремих mediasource ці значення потрібні не будуть.

In [9]:
romi_daily = merged_data.copy().groupby(
    ['media_source', 'date']
).agg({'earnings': 'sum', 'cost': 'sum'}).reset_index()
romi_daily = romi_daily[romi_daily['media_source'] != "Organic/Unknown"]
romi_daily.head()

,media_source,date,earnings,cost
0,ByteDance,2020-07-09,0.0,0.00
1,ByteDance,2020-07-22,0.5,0.00
2,ByteDance,2020-07-23,0.0,0.00
3,ByteDance,2020-07-29,0.0,101.00
4,ByteDance,2020-07-30,0.0,43.28


Зберігаємо датафрейм у csv файл і проводимо подальші розрахунки в Tableau.

In [10]:
romi_daily.to_csv('output_files/romi_daily.csv', index=False)

#### ROMI поденно у розрізі топ-10 країн (топ вибираємо по витратах (cost) на маркетинг за весь час)

Підраховуємо витрати за кожною країною і знаходимо топ-10.

In [11]:
country_expenses = merged_data.copy().groupby(
    'country_code'
).agg({'cost': 'sum'}).reset_index()

top_10_countries = country_expenses.sort_values(by='cost', ascending=False).head(10)
top_10_countries

,country_code,cost
225,US,209215.10
167,Organic/Unknown,47253.65
186,RU,18326.18
30,BR,17242.87
36,CA,13629.16
152,MX,9470.44
74,GB,8777.38
92,HK,7999.07
174,PL,7094.68
54,DE,6987.01


Фільтруємо дані за топ-10 країнами, групуємо за media_source, date і country_code та обчислюємо earnings і cost для кожної групи.

In [12]:
daily_top10_countries = merged_data[merged_data['country_code'].isin(top_10_countries['country_code'])]

daily_top10_countries = daily_top10_countries.groupby(
    ['media_source', 'date', 'country_code']
).agg({'earnings': 'sum', 'cost': 'sum'}).reset_index()
daily_top10_countries

,media_source,date,country_code,earnings,cost
0,ByteDance,2020-07-09,US,0.00,0.00
1,ByteDance,2020-07-22,US,0.50,0.00
2,ByteDance,2020-07-23,US,0.00,0.00
3,ByteDance,2020-07-29,US,0.00,101.00
4,ByteDance,2020-07-30,US,0.00,43.28
...,...,...,...,...,...
1398,Organic/Unknown,2020-09-26,Organic/Unknown,586.94,0.00
1399,Organic/Unknown,2020-09-27,Organic/Unknown,578.93,0.00
1400,Organic/Unknown,2020-09-28,Organic/Unknown,704.82,0.00
1401,Organic/Unknown,2020-09-29,Organic/Unknown,724.34,0.00


Зберігаємо датафрейм у csv файл і проводимо подальші розрахунки в Tableau.

In [13]:
daily_top10_countries.to_csv('output_files/daily_top10_countries.csv', index=False)

#### Топ-10 комбінацій mediasource та country з найвищим ROMI для кожного календарного місяця

Додаємо стовпець місяця, групуємо дані за media_source, country_code, month, та обчислюємо earnings та cost.

In [14]:
monthly_data = merged_data.copy()
monthly_data['month'] = monthly_data['date'].dt.to_period('M')

monthly_agg = monthly_data.copy().groupby(
    ['media_source', 'country_code', 'month']
).agg({'earnings': 'sum', 'cost': 'sum'}).reset_index()
monthly_agg

,media_source,country_code,month,earnings,cost
0,ByteDance,AE,2020-08,0.00,3.32
1,ByteDance,AE,2020-09,0.00,193.85
2,ByteDance,AU,2020-08,0.00,0.01
3,ByteDance,AU,2020-09,0.00,206.43
4,ByteDance,CA,2020-08,0.00,4.81
...,...,...,...,...,...
984,Google Ads,ZW,2020-08,0.00,0.00
985,Google Ads,ZW,2020-09,11.99,0.00
986,Organic/Unknown,Organic/Unknown,2020-07,12661.34,0.00
987,Organic/Unknown,Organic/Unknown,2020-08,19697.42,0.00


Сортуємо дані для коректного обчислення кумулятивних сум earnings і cost і обчислюємо кумулятивний ROMI.

In [15]:
monthly_agg = monthly_agg.sort_values(by=['media_source', 'country_code', 'month'])

monthly_agg['cumulative_earnings'] = monthly_agg.groupby(['media_source', 'country_code'])['earnings'].cumsum()
monthly_agg['cumulative_cost'] = monthly_agg.groupby(['media_source', 'country_code'])['cost'].cumsum()

monthly_agg['cumulative_ROMI'] = ((monthly_agg['cumulative_earnings'] - monthly_agg['cumulative_cost']) 
                                  / monthly_agg['cumulative_cost']) * 100
monthly_agg

,media_source,country_code,month,earnings,cost,cumulative_earnings,cumulative_cost,cumulative_ROMI
0,ByteDance,AE,2020-08,0.00,3.32,0.00,3.32,-100.0
1,ByteDance,AE,2020-09,0.00,193.85,0.00,197.17,-100.0
2,ByteDance,AU,2020-08,0.00,0.01,0.00,0.01,-100.0
3,ByteDance,AU,2020-09,0.00,206.43,0.00,206.44,-100.0
4,ByteDance,CA,2020-08,0.00,4.81,0.00,4.81,-100.0
...,...,...,...,...,...,...,...,...
984,Google Ads,ZW,2020-08,0.00,0.00,0.00,0.00,NaN
985,Google Ads,ZW,2020-09,11.99,0.00,11.99,0.00,inf
986,Organic/Unknown,Organic/Unknown,2020-07,12661.34,0.00,12661.34,0.00,inf
987,Organic/Unknown,Organic/Unknown,2020-08,19697.42,0.00,32358.76,0.00,inf


Редагуємо некоректні дані і оибираємо топ-10 комбінацій media_source та country_code кожного місяця за cumulative_ROMI.

In [16]:
monthly_agg['cumulative_ROMI'] = monthly_agg['cumulative_ROMI'].fillna(0)
monthly_agg.loc[monthly_agg['cumulative_cost'] == 0, 'cumulative_ROMI'] = 0

monthly_top10_cumulative_romi = monthly_agg.sort_values(
    ['month', 'cumulative_ROMI'], ascending=[True, False]
).groupby('month').head(10)

monthly_top10_cumulative_romi

,media_source,country_code,month,earnings,cost,cumulative_earnings,cumulative_cost,cumulative_ROMI
553,Facebook Ads,PG,2020-07,89.45,18.45,89.45,18.45,384.823848
281,Facebook Ads,GI,2020-07,49.99,11.99,49.99,11.99,316.930776
86,Facebook Ads,AW,2020-07,49.99,12.68,49.99,12.68,294.242902
643,Facebook Ads,SR,2020-07,30.97,9.68,30.97,9.68,219.938017
123,Facebook Ads,BM,2020-07,70.97,23.33,70.97,23.33,204.200600
225,Facebook Ads,EE,2020-07,158.92,53.67,158.92,53.67,196.105832
272,Facebook Ads,GF,2020-07,49.99,19.97,49.99,19.97,150.325488
589,Facebook Ads,RE,2020-07,70.47,29.04,70.47,29.04,142.665289
132,Facebook Ads,BQ,2020-07,10.49,4.53,10.49,4.53,131.567329
326,Facebook Ads,HT,2020-07,72.43,33.74,72.43,33.74,114.671014


Зберігаємо датафрейм у csv файл і проводимо подальші розрахунки в Tableau.

In [17]:
monthly_top10_cumulative_romi.to_csv('output_files/monthly_top10_cumulative_romi.csv', index=False)

Після аналізу дашборда можна сказати, що бізнес поки що не окупається, але показники ROMI в більшості випадків збільшуються з часом. Найкращу динаміку позитивної зміни ROMI показала платформа Facebook Ads, також саме у неї найбільша кількість позитивних ROMI за країнами. Платформа Google Ads же навпаки погіршує ROMI з часом, тому можна зробити висновок що у Facebook Ads були зроблені найбільш ефективні вкладення.